### Import


In [ ]:
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, classification_report

from sklearn.model_selection import train_test_split

import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from cnn_classifier import CNN

import pandas as pd
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

import time



### Load dataset

In [ ]:
dataset = np.load("dataset.npz")
X, y = dataset["X"], dataset["y"]

In [ ]:
k=10
shape = (X[k].reshape(20,20))
flatten = shape.flatten()

print(f"Label for image {k}: {y[k]}")
plt.imshow(shape, vmin=0, vmax=255, cmap="gray")
plt.show()

In [ ]:
random_state = 42

X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=random_state
)

X_trainval_tensor = torch.tensor(X_trainval, dtype=torch.float32)
y_trainval_tensor = torch.tensor(y_trainval, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

print("Unique labels:", np.unique(y_trainval))

## Class distribution

In [ ]:
def class_dist(y):
    unique, counts = np.unique(y, return_counts=True)
    plt.bar(unique, counts)
    plt.xlabel("Class")
    plt.ylabel("Count")
    plt.title("Class distribution")
    plt.show()
class_dist(y_trainval)

In [ ]:
def blurred():
    means = []
    for label in range(15):
        means.append(X_trainval[y_trainval == label].mean(axis=0))

    fig, axes = plt.subplots(3, 5, figsize=(8, 5))
    for i, ax in enumerate(axes.flat):
        ax.imshow(means[i].reshape(20, 20), cmap='gray')
        ax.set_title(f"Class {i}")
        ax.axis('off')
    plt.show()

blurred()

# CNN

In [ ]:
pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('cnn', CNN())
        ])

param_grid = {
   "cnn__learning_rate": [0.001, 0.0005],
   "cnn__dropout_rate": [0.25, 0.5],
   "cnn__batch_size": [32, 64],
   "cnn__weight_decay": [0, 1e-4],
   "cnn__random_state": [random_state]
}

grid = GridSearchCV(pipeline, param_grid, cv=3, scoring='f1_weighted', n_jobs=-1, verbose=3)
grid.fit(X_trainval, y_trainval)

print("Best parameters:", grid.best_params_)

In [ ]:
best_model = grid.best_estimator_["cnn"]
best_model_scaler = grid.best_estimator_["scaler"]

In [ ]:
X_test = best_model_scaler.transform(X_test)
test_acc = best_model.compute_accuracy(X_test, y_test)
y_pred = best_model.predict(X_test)
test_f1 = f1_score(y_test, y_pred, average='weighted')

print(f"\nTest Accuracy: {test_acc * 100:.2f}%")
print(f"Weighted F1-score: {test_f1:.4f}")
print(classification_report(y_test, y_pred))

##### Plotting Training Loss

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(best_model.train_loss_history, label='Training Loss')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Training Loss Curve')
plt.legend()
plt.show()

# KNN

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

param_grid_knn = {
    'classifier__n_neighbors': [2, 3, 4, 5, 6, 7],
    'classifier__metric': ['euclidean', 'manhattan', 'minkowski', 'chebyshev']
}

param_grid_pca = {
    'pca__n_components': [0.70, 0.80, 0.90, 0.95]
}

def choose_pipeline(classifier, pca=False):
    if pca:
        return Pipeline([
            ('scaler', StandardScaler()),
            ('pca', PCA(n_components=0.95)),
            ('classifier', classifier)
        ])
    else:
        return Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', classifier)
        ])

def knn_function(pca=False):
    pipeline = choose_pipeline(KNeighborsClassifier(), pca)
    if pca:
        param_grid_knn.update(param_grid_pca)
        
    grid_knn = GridSearchCV(pipeline, param_grid_knn, cv=3, scoring='f1_weighted', n_jobs=-1)
    time_start = time.time()
    grid_knn.fit(x_train, y_train)
    time_elapsed_training = time.time() - time_start

    best_model = grid_knn.best_estimator_
    
    time_start = time.time()
    y_pred = grid_knn.predict(x_test) 
    time_elapsed_prediction = time.time() - time_start
    
    score = grid_knn.score(x_test, y_test)
    
    print(f"KNN training time: {time_elapsed_training:.4f} seconds")
    print(f"KNN prediction time: {time_elapsed_prediction:.4f} seconds")
    print(f"KNN Classifier score: {score * 100:.2f}%") 
    
    return best_model, y_pred, grid_knn, score, time_elapsed_training, time_elapsed_prediction

#### Running KNN Problem 1

In [ ]:
best_model_knn, y_pred, grid_knn, score, train_time, pred_time = knn_function(pca=False) 

##### Vizualising for KNN

In [ ]:
results = grid_knn.cv_results_
scores = results['mean_test_score']
params = results['params']

df = pd.DataFrame(params)
df['score'] = scores

pivot = df.pivot(index='n_neighbors', columns='metric', values='score')

bubble_to_hot_pink = LinearSegmentedColormap.from_list(
    "bubble_to_hot_pink",
    ["#fec1d9", "#ffa6cb", "#ff82c3", "#ff0080ca"], 
    N=256 
)

plt.figure(figsize=(8, 6))
sns.heatmap(pivot, annot=True, fmt=".3f", cmap=bubble_to_hot_pink)
plt.title("KNN Grid Search Mean F1 Score", fontsize=14)
plt.ylabel("n_neighbors")
plt.xlabel("metric")
plt.show()


In [ ]:
misclassified = np.where(y_pred != y_test)[0]

for idx in misclassified[:10]:
    img = x_test[idx].reshape(20, 20) * 255 
    plt.imshow(img, vmin=0, vmax=255, cmap="gray")
    plt.title(f"True: {y_test[idx]}, Predicted: {y_pred[idx]}")
    plt.show()

# Problem 2: running KNN with PCA

In [ ]:
best_model_knn_pca, y_pred_pca, grid_knn_pca, score_pca, train_time_pca, pred_time_pca = knn_function(pca=True)

##### Plotting score, training time and prediction time for KNN with and without PCA

In [ ]:
models = ['KNN (no PCA)', 'KNN + PCA']
scores = [score, score_pca]
train_times = [train_time, train_time_pca]
pred_times = [pred_time, pred_time_pca]

#Score comparison
plt.figure(figsize=(6,4))
plt.bar(models, scores, color=["#0F8FE3", '#48C9B0'])
plt.ylabel('Score')
plt.title('KNN Score Comparison')
for i, v in enumerate(scores):
    plt.text(i, v + 0.01, f"{v*100:.2f}%", ha='center')
plt.ylim(0, 1)
plt.show()

# Training time comparison
plt.figure(figsize=(6,4))
plt.bar(models, train_times, color=["#FF8CDE", "#FF1884"])
plt.ylabel('Training Time (s)')
plt.title('KNN Training Time Comparison')
for i, v in enumerate(train_times):
    plt.text(i, v + 0.02, f"{v:.2f}s", ha='center')
plt.show()

# Prediction time comparison
plt.figure(figsize=(6,4))
plt.bar(models, pred_times, color=["#FFED64", "#FB811D"])
plt.ylabel('Prediction Time (s)')
plt.title('KNN Prediction Time Comparison')
for i, v in enumerate(pred_times):
    plt.text(i, v + 0.002, f"{v:.4f}s", ha='center')
plt.show()


# Problem 3

In [ ]:
corrupted_dataset = np.load("dataset_corrupted.npz")
X_corrupted = corrupted_dataset["X"] / 255
X_corrupted_tensor = torch.tensor(X_corrupted, dtype=torch.float32).view(-1, 1, 20, 20)
X_corrupted_tensor = X_corrupted_tensor.to(best_model.device)


best_model.model.eval()
with torch.no_grad():
    outputs = best_model.model(X_corrupted_tensor)  

max_probs = outputs.max(dim=1).values

threshold = 0.9

ood_indices = (max_probs < threshold).nonzero(as_tuple=True)[0].cpu().numpy()
print(f"Found {len(ood_indices)} suspicious/corrupted images")

##### Vizualising corrupted images

In [ ]:
num_images = min(116, len(ood_indices))
grid_size = int(num_images**0.5)

fig, axes = plt.subplots(grid_size, grid_size+1, figsize=(15, 15))
fig.suptitle("Suspicious/Corrupted Images", fontsize=16)

for i, ax in enumerate(axes.flat):
    if i < num_images:
        idx = ood_indices[i]
        ax.imshow(corrupted_dataset["X"][idx].reshape(20, 20), cmap="gray")
    ax.axis('off')

plt.tight_layout()
plt.show()